Muhammad Imran Wong bin Muhammad Khalid Wong
IS01082642

In [3]:
!pip install capsolver

In [5]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random
import capsolver  # Using CapSolver to bypass CAPTCHA

# Provide your CapSolver API key
capsolver.api_key = "YOUR_CAPSOLVER_API_KEY"

# Initialize Chrome with specific options
chrome_settings = uc.ChromeOptions()
chrome_settings.add_argument("--no-sandbox")
chrome_settings.add_argument("--disable-gpu")

# Launch the browser
browser = uc.Chrome(options=chrome_settings)

# Product link (Novencci Sneakers)
product_url = "https://www.lazada.com.my/products/novencci-unisex-mens-womens-outdoor-sneakers-sport-i3567542617-s19976126346.html"

# Open product page
browser.get(product_url)

# Random sleep to mimic human interaction
time.sleep(random.uniform(6, 12))

# A place to collect review data
collected_reviews = []

# Set how many pages you want to scrape
pages_to_scrape = 5

# CAPTCHA handler function
def bypass_captcha():
    print("CAPTCHA found! Trying to bypass with CapSolver...")
    try:
        # Attempt to solve CAPTCHA
        result = capsolver.solve({
            "type": "ReCaptchaV2TaskProxyless",
            "websiteURL": browser.current_url,
            "websiteKey": "SITE_KEY_HERE"  # Update with actual site key if known
        })
        print(f"CAPTCHA successfully solved: {result}")
        time.sleep(8)  # Give time for CAPTCHA processing
        return result
    except Exception as error:
        print(f"Failed to bypass CAPTCHA: {error}")
        return None

# Start scraping reviews
for current_page in range(1, pages_to_scrape + 1):
    print(f"\n[INFO] Collecting reviews on page {current_page}...")

    # Scroll to bottom to trigger review load
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(5, 9))

    # Get reviews on current page
    review_items = browser.find_elements(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[1]/div')

    if not review_items:
        print("[WARNING] No reviews found! Checking for CAPTCHA...")
        captcha_result = bypass_captcha()
        if captcha_result:
            time.sleep(random.uniform(6, 12))
            continue
        else:
            print("[ERROR] Unable to proceed without solving CAPTCHA. Exiting.")
            break

    # Extract info from each review
    for item in review_items:
        try:
            user = item.find_element(By.XPATH, ".//div[@class='middle']/span").text.strip()
        except:
            user = "Anonymous"

        try:
            review_date = item.find_element(By.XPATH, ".//span[@class='title right']").text.strip()
        except:
            review_date = "No Date Provided"

        try:
            review_text = item.find_element(By.XPATH, ".//div[@class='content']").text.strip()
        except:
            review_text = "No Review Text"

        # Append the extracted review data
        collected_reviews.append([user, review_date, review_text])

    print(f"[INFO] Page {current_page} complete. Total reviews collected: {len(collected_reviews)}")

    # Navigate to next review page
    try:
        pagination_block = browser.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[2]/div/div')
        next_btn = pagination_block.find_element(By.XPATH, f".//button[contains(@class, 'next-pagination-item') and text()='{current_page + 1}']")

        if next_btn:
            print(f"[ACTION] Moving to page {current_page + 1}...")
            browser.execute_script("arguments[0].scrollIntoView();", next_btn)
            time.sleep(random.uniform(3, 5))
            browser.execute_script("window.scrollBy(0, -120);")
            time.sleep(random.uniform(2, 3))

            # Click using JS to avoid click interception issues
            browser.execute_script("arguments[0].click();", next_btn)
            time.sleep(random.uniform(6, 10))

            # Wait until page reviews refresh
            WebDriverWait(browser, 20).until(EC.staleness_of(review_items[0]))
        else:
            print("[INFO] No further pages detected. Scraping complete.")
            break

    except Exception as nav_error:
        print(f"[ERROR] Failed to navigate to next page: {nav_error}")
        break

# Close the browser session
browser.quit()

# Create DataFrame and export reviews to CSV
review_df = pd.DataFrame(collected_reviews, columns=["User", "Date", "Review"])
review_df.to_csv("novencci_sneakers_reviews.csv", index=False, encoding="utf-8")

print(f"\n[SUCCESS] Scraping finished. {len(collected_reviews)} reviews saved to 'novencci_sneakers_reviews.csv'.")



[INFO] Collecting reviews on page 1...
[INFO] Page 1 complete. Total reviews collected: 5
[ACTION] Moving to page 2...

[INFO] Collecting reviews on page 2...
[INFO] Page 2 complete. Total reviews collected: 10
[ACTION] Moving to page 3...

[INFO] Collecting reviews on page 3...
[INFO] Page 3 complete. Total reviews collected: 15
[ACTION] Moving to page 4...

[INFO] Collecting reviews on page 4...
[INFO] Page 4 complete. Total reviews collected: 20
[ACTION] Moving to page 5...

[INFO] Collecting reviews on page 5...
[INFO] Page 5 complete. Total reviews collected: 25
[ACTION] Moving to page 6...

[SUCCESS] Scraping finished. 25 reviews saved to 'novencci_sneakers_reviews.csv'.
